Purpose of this notebook is to prepare a database for the stored procedure scripts and ultimately the orchestration.ipynb. Each script expects normalized table names ex. consolidated_mem_type, mem_type_new_import, etc.

In [1]:
import os
import re
import datetime
import itertools
import json
import sqlalchemy

In [2]:
from container_credentials import return_credentials

In [3]:
# DEFINE THE DATABASE CREDENTIALS
cred_dict = return_credentials()

user = cred_dict['user'] 
password = cred_dict['pass'] 
host = cred_dict['host'] 
port = cred_dict['port'] 
database = cred_dict['database'] 

def get_connection():
	return sqlalchemy.create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		engine = get_connection()
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)


Connection to the 172.17.0.2 for user root created successfully.


In [ ]:
'''
DROP TABLE consolidated_mem_status;
CREATE TABLE consolidated_mem_status AS SELECT * FROM mem_status_1007_ts;

DROP TABLE consolidated_mem_type;
CREATE TABLE consolidated_mem_type AS SELECT * FROM mem_type_1007_ts;

DROP TABLE mem_status_new_import;
CREATE TABLE mem_status_new_import AS SELECT * FROM mem_status_1007_ts;

DROP TABLE mem_type_new_import;
CREATE TABLE mem_type_new_import AS SELECT * FROM mem_type_1007_ts;
'''

In [5]:
#dictionary of instructions
instruct_dicts = [{'table_name':'consolidated_mem_status','source':'mem_status_1007_ts'},
                 {'table_name':'consolidated_mem_type','source':'mem_type_1007_ts'},
                 {'table_name':'mem_status_new_import','source':'mem_status_1013_ts'},
                 {'table_name':'mem_type_new_import','source':'mem_type_1013_ts'}]

In [14]:
for i in instruct_dicts:
    print(i['table_name'])

consolidated_mem_status
consolidated_mem_type
mem_status_new_import
mem_type_new_import


In [15]:
with engine.connect() as conn: #interacting w/db through Connection class
    for i in instruct_dicts:
        conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS {i['table_name']}"))
        conn.execute(sqlalchemy.text(f"CREATE TABLE {i['table_name']} AS SELECT * FROM {i['source']}"))

In [16]:
#verify the above
#inspector option: https://docs.sqlalchemy.org/en/20/core/reflection.html#fine-grained-reflection-with-inspector
# inspector is a low level interface which provides a backend-agnostic system of loading lists of schema, table, column, and constraint descriptions from a given database is also available.
from sqlalchemy import inspect
insp = inspect(engine)
table_name_list = insp.get_table_names()

#gather all the table names from the above dict:
lista = [i['table_name'] for i in instruct_dicts]

#determine whether the tables I expect to have been injected into db from the civiActivityReport.ipynb are there
if all([i in table_name_list for i in(lista)]):
    print('all clear to proceed')
else:
    print('new tables from copy_rename() step aren\'t found in db')


all clear to proceed
